In [1]:
import json
from tensorflow.keras import optimizers
import numpy as np

In [2]:
data = []
endata = []
chdata = []

def chine(string):
    for ch in string :
        if u'\u4e00' <= ch <= u'\u9fff':
            return True
    return False

for line in open('data.json', 'r', encoding='utf-8'):
    data.append(json.loads(line))  


for i in range(len(data)):
    if chine(data[i]['chinese']) == True :
        if len(data[i]['chinese']) < 10:
            endata.append(data[i]['english'])
            chdata.append('@'+data[i]['chinese']+'。')
        if len(chdata) == 100 :
            break

en_vocab = set(''.join(endata))
id2en = list(en_vocab)
en2id = {c:i for i,c in enumerate(id2en)}

# 分別生成中英文字典
ch_vocab = set(''.join(chdata))
id2ch = list(ch_vocab)
ch2id = {c:i for i,c in enumerate(id2ch)}

print('英文數據:\n', endata)
print('\n中文數據:\n', chdata)
print('\n英文字典:\n', en2id)
print('\n中文字典共計\n:', ch2id)


# en_vocab = set(''.join(endata))
# id2en = list(en_vocab)
# en2id = {c:i for i,c in enumerate(id2en)}

英文數據:
 ['Look at these coasters over here.', 'Choose a recorder.', "I hadn't paid the telephone bill.", "That's easier said than done, of course.", 'Side-to-Side Movements.', 'about like 80 degrees.', 'We all are from Shandong.', 'She was possessed by a devil.', 'This wool knits up well.', 'The majority was wrong last time.', 'Stone Soup Stories to Go!', 'Done. See you tomorrow.', 'He eased some of the strains on the poor.', 'Could it be that it was written wrongly?', 'What a terrible temper!', 'Great talents flower late.', 'I forbid you to make a sortie today.', 'C：My surname is Jiang.', 'Well, if it was greater .', 'They looked over to the left.', 'To supervise the management of printing industry.', 'no one else can see you shake your head.', 'All photos dials.', 'Stained glass window panels;', 'The murderer was caught red-handed.', 'You don’t love Melanie.', 'His style is very lucid .', 'At this time, there was a male cat.', 'The truth has leaked out.', 'Material evidence must also 

In [3]:
en_num_data = [[en2id[en] for en in line ] for line in endata]
ch_num_data = [[ch2id[ch] for ch in line] for line in chdata]
de_num_data = [[ch2id[ch] for ch in line][1:] for line in chdata]

print('char:', endata[1])
print('index:', en_num_data[1])
print(len(en_num_data))
len(ch_num_data)
# de_num_data

char: Choose a recorder.
index: [53, 55, 25, 25, 56, 34, 37, 33, 37, 7, 34, 0, 25, 7, 30, 34, 7, 6]
100


100

In [4]:
import numpy as np

max_encoder_seq_length = max([len(txt) for txt in en_num_data])
max_decoder_seq_length = max([len(txt) for txt in ch_num_data])
print('max encoder length:', max_encoder_seq_length)
print('max decoder length:', max_decoder_seq_length)

# 將數據進行onehot處理
encoder_input_data = np.zeros((len(en_num_data), max_encoder_seq_length, len(en2id)), dtype='float32')
decoder_input_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')
decoder_target_data = np.zeros((len(ch_num_data), max_decoder_seq_length, len(ch2id)), dtype='float32')

for i in range(len(ch_num_data)):
    for t, j in enumerate(en_num_data[i]):
        encoder_input_data[i, t, j] = 1.
    for t, j in enumerate(ch_num_data[i]):
        decoder_input_data[i, t, j] = 1.
    for t, j in enumerate(de_num_data[i]):
        decoder_target_data[i, t, j] = 1.

print('index data:\n', en_num_data[1])
print('one hot data:\n', encoder_input_data[1])

max encoder length: 49
max decoder length: 11
index data:
 [53, 55, 25, 25, 56, 34, 37, 33, 37, 7, 34, 0, 25, 7, 30, 34, 7, 6]
one hot data:
 [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [5]:
EN_VOCAB_SIZE = len(en2id)
CH_VOCAB_SIZE = len(ch2id)
HIDDEN_SIZE = 256

LEARNING_RATE = 0.01
BATCH_SIZE = 20
EPOCHS = 250

In [6]:
# ======================================keras model==================================
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from tensorflow.keras.optimizers import Adam
import numpy as np

# ==============encoder=============
encoder_inputs = Input(shape=(None, EN_VOCAB_SIZE))
#emb_inp = Embedding(output_dim=HIDDEN_SIZE, input_dim=EN_VOCAB_SIZE)(encoder_inputs)
encoder_h1, encoder_state_h1, encoder_state_c1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)(encoder_inputs)
encoder_h2, encoder_state_h2, encoder_state_c2 = LSTM(HIDDEN_SIZE, return_state=True)(encoder_h1)

In [7]:
decoder_inputs = Input(shape=(None, CH_VOCAB_SIZE))

#emb_target = Embedding(output_dim=HIDDEN_SIZE, input_dim=CH_VOCAB_SIZE, mask_zero=True)(decoder_inputs)
lstm1 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
lstm2 = LSTM(HIDDEN_SIZE, return_sequences=True, return_state=True)
decoder_dense = Dense(CH_VOCAB_SIZE, activation='softmax')

decoder_h1, _, _ = lstm1(decoder_inputs, initial_state=[encoder_state_h1, encoder_state_c1])
decoder_h2, _, _ = lstm2(decoder_h1, initial_state=[encoder_state_h2, encoder_state_c2])
decoder_outputs = decoder_dense(decoder_h2)

In [11]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
opt = Adam(lr =LEARNING_RATE, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=BATCH_SIZE,
          epochs=EPOCHS,
          validation_split=0.)

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 66)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 403)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  330752      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  675840      input_2[0][0]                    
                                                                 lstm[0][1]                 

5/5 [==============================] - 1s 294ms/step - loss: 0.2027 - accuracy: 0.7200
Epoch 61/250
5/5 [==============================] - 1s 285ms/step - loss: 0.2034 - accuracy: 0.7273
Epoch 62/250
5/5 [==============================] - 1s 278ms/step - loss: 0.2038 - accuracy: 0.7200
Epoch 63/250
5/5 [==============================] - 1s 286ms/step - loss: 0.2018 - accuracy: 0.7245
Epoch 64/250
5/5 [==============================] - 1s 279ms/step - loss: 0.2030 - accuracy: 0.7200
Epoch 65/250
5/5 [==============================] - 1s 288ms/step - loss: 0.2015 - accuracy: 0.7218
Epoch 66/250
5/5 [==============================] - 1s 286ms/step - loss: 0.1985 - accuracy: 0.7264
Epoch 67/250
5/5 [==============================] - 2s 303ms/step - loss: 0.1997 - accuracy: 0.7264
Epoch 68/250
5/5 [==============================] - 1s 297ms/step - loss: 0.1990 - accuracy: 0.7255
Epoch 69/250
5/5 [==============================] - 1s 284ms/step - loss: 0.2034 - accuracy: 0.7209
Epoch 70/250


5/5 [==============================] - 1s 265ms/step - loss: 0.1453 - accuracy: 0.7382
Epoch 142/250
5/5 [==============================] - 1s 263ms/step - loss: 0.1485 - accuracy: 0.7409
Epoch 143/250
5/5 [==============================] - 1s 271ms/step - loss: 0.1456 - accuracy: 0.7373
Epoch 144/250
5/5 [==============================] - 1s 276ms/step - loss: 0.1451 - accuracy: 0.7391
Epoch 145/250
5/5 [==============================] - 2s 314ms/step - loss: 0.1438 - accuracy: 0.7409
Epoch 146/250
5/5 [==============================] - 1s 303ms/step - loss: 0.1424 - accuracy: 0.7409
Epoch 147/250
5/5 [==============================] - 1s 268ms/step - loss: 0.1468 - accuracy: 0.7382
Epoch 148/250
5/5 [==============================] - 1s 266ms/step - loss: 0.1393 - accuracy: 0.7382
Epoch 149/250
5/5 [==============================] - 1s 273ms/step - loss: 0.1385 - accuracy: 0.7409
Epoch 150/250
5/5 [==============================] - 1s 269ms/step - loss: 0.1412 - accuracy: 0.7382
Epoc

5/5 [==============================] - 1s 279ms/step - loss: 0.5145 - accuracy: 0.6700
Epoch 223/250
5/5 [==============================] - 1s 289ms/step - loss: 0.4967 - accuracy: 0.6791
Epoch 224/250
5/5 [==============================] - 2s 339ms/step - loss: 0.5151 - accuracy: 0.6655
Epoch 225/250
5/5 [==============================] - 2s 315ms/step - loss: 0.3939 - accuracy: 0.6836
Epoch 226/250
5/5 [==============================] - 2s 304ms/step - loss: 0.3457 - accuracy: 0.6955
Epoch 227/250
5/5 [==============================] - 2s 373ms/step - loss: 0.3173 - accuracy: 0.7009
Epoch 228/250
5/5 [==============================] - 1s 282ms/step - loss: 0.2768 - accuracy: 0.7064
Epoch 229/250
5/5 [==============================] - 2s 351ms/step - loss: 0.2523 - accuracy: 0.7182
Epoch 230/250
5/5 [==============================] - 1s 291ms/step - loss: 0.2344 - accuracy: 0.7209
Epoch 231/250
5/5 [==============================] - 2s 301ms/step - loss: 0.2358 - accuracy: 0.7182
Epoc

In [12]:
encoder_model = Model(encoder_inputs, [encoder_state_h1, encoder_state_c1, encoder_state_h2, encoder_state_c2])

# 預測模型中的decoder的初始化狀態需要傳入新的狀態
decoder_state_input_h1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c1 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_h2 = Input(shape=(HIDDEN_SIZE,))
decoder_state_input_c2 = Input(shape=(HIDDEN_SIZE,))

# 使用傳入的值來初始化當前模型的輸入狀態
decoder_h1, state_h1, state_c1 = lstm1(decoder_inputs, initial_state=[decoder_state_input_h1, decoder_state_input_c1])
decoder_h2, state_h2, state_c2 = lstm2(decoder_h1, initial_state=[decoder_state_input_h2, decoder_state_input_c2])
decoder_outputs = decoder_dense(decoder_h2)

decoder_model = Model([decoder_inputs, decoder_state_input_h1, decoder_state_input_c1, decoder_state_input_h2, decoder_state_input_c2], 
                      [decoder_outputs, state_h1, state_c1, state_h2, state_c2])

In [13]:
for k in range(0,100):
    test_data = encoder_input_data[k:k+1]
    h1, c1, h2, c2 = encoder_model.predict(test_data)
    target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
    target_seq[0, 0, ch2id['@']] = 1
    outputs = []
    while True:
        output_tokens, h1, c1, h2, c2 = decoder_model.predict([target_seq, h1, c1, h2, c2])
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        outputs.append(sampled_token_index)
        target_seq = np.zeros((1, 1, CH_VOCAB_SIZE))
        target_seq[0, 0, sampled_token_index] = 1
        if sampled_token_index == ch2id['。']: break
        # if len(outputs) > 11: break
    
    print(endata[k])
    print(''.join([id2ch[i] for i in outputs]))

Look at these coasters over here.
看看这边的杯垫。
Choose a recorder.
莫非写错了?。
I hadn't paid the telephone bill.
我给你指路。
That's easier said than done, of course.
这可是件大事儿啊！。
Side-to-Side Movements.
侧向运动。
about like 80 degrees.
大概华氏80度吧。
We all are from Shandong.
如果拉力很大。
She was possessed by a devil.
我应该到屋顶上吗？。
This wool knits up well.
这可是件大事儿啊！。
The majority was wrong last time.
这可是件大事儿啊！。
Stone Soup Stories to Go!
石头汤（故事去！）。
Done. See you tomorrow.
我给你指路。
He eased some of the strains on the poor.
他是一个初中生吗？。
Could it be that it was written wrongly?
考虑你的听众。
What a terrible temper!
脾气真够坏的！。
Great talents flower late.
格温：让我来看看。
I forbid you to make a sortie today.
我给你指路。
C：My surname is Jiang.
莫非写错了?。
Well, if it was greater .
如果拉力很大。
They looked over to the left.
这可是件大事儿啊！。
To supervise the management of printing industry.
这可是件大事儿啊！。
no one else can see you shake your head.
没人看的见你。
All photos dials.
阒无一人。
Stained glass window panels;
石头汤（故事去！）。
The murderer was caught red-handed.
这可是件大事儿啊！。
You don